In [249]:
import time
from datetime import datetime, date, timedelta
import pandas as pd 
import requests
import time
import pickle

start_time = time.time()


def get_daterange(start_date, end_date, step):
    while start_date <= end_date:
        yield start_date
        start_date += step

def p2f(x):
    return float(x.strip('%'))/100

def windconversion(wind):
    return wind / 0.44704

def timeconversion(entry):
    utc_time = datetime.strptime(entry[:19],'%Y-%m-%dT%H:%M:%S')
    return pd.to_datetime(utc_time)

def seasonconversion(timestamp):
    if (timestamp > datetime(2018, 12, 21)) & (timestamp <= datetime(2019, 3, 20) ):
        season = 1
    elif (timestamp > datetime(2019, 3, 20))& (timestamp <= datetime(2019, 6, 21)):
        season = 2
    elif (timestamp > datetime(2019, 6, 21))& (timestamp <= datetime(2019, 9, 23) ):
        season = 3
    elif (timestamp > datetime(2019, 9, 23))& (timestamp <= datetime(2019, 12, 21)):
        season = 4  
    elif (timestamp > datetime(2019, 12, 21))& (timestamp <= datetime(2020, 3, 20)):
        season = 1
    elif (timestamp > datetime(2020, 3, 20))& (timestamp <= datetime(2020, 6, 20)):
        season = 2
    elif (timestamp > datetime(2020, 6, 20))& (timestamp <= datetime(2020, 9, 22) ):
        season = 3
    elif (timestamp > datetime(2020, 9, 22))& (timestamp <=  datetime(2020, 12, 21)):
        season = 4   
    elif (timestamp > datetime(2020, 12, 21))& (timestamp <= datetime(2021, 3, 20)):
        season = 1
    elif (timestamp > datetime(2021, 3, 20))& (timestamp <= datetime(2021, 6, 20)):
        season = 2
    elif (timestamp > datetime(2021, 6, 20))& (timestamp <= datetime(2021, 9, 22) ):
        season = 3
    elif (timestamp > datetime(2021, 9, 22))& (timestamp <= datetime(2021, 12, 21)):
        season = 4 
    else:
        season = None
    return season


DistrictLocation= pd.DataFrame({'District' : ['BC','CM','EA','ME','NE','OC','RA'],'Longitude':[32.793616,32.599472,32.769567,33.230294,32.997382,33.432239,33.126757],'Latitude':[-117.174505,-117.02563,-116.862862,-117.368176,-117.050991,-117.621563,-116.212305]})

Assume SDGE LFM Dataset is stored in S3 location

In [250]:
import pandas as pd
import boto3
import glob
from statsmodels.tsa.arima.model import ARIMA
client = boto3.client('s3')
path = 's3://demo-primary-dp/LFM.csv'
LFM = pd.read_csv(path,parse_dates=['Date'])
LFM['ME'] = LFM['ME'].apply(lambda x: p2f(x))
LFM['RA'] = LFM['RA'].apply(lambda x: p2f(x))
LFM['EA'] = LFM['EA'].apply(lambda x: p2f(x))
LFM['NE'] = LFM['NE'].apply(lambda x: p2f(x))
LFM['OC'] = LFM['OC'].apply(lambda x: p2f(x))
LFM['NC'] = LFM['NC'].apply(lambda x: p2f(x))
LFM['BC'] = LFM['BC'].apply(lambda x: p2f(x))
LFM['CM'] = LFM['CM'].apply(lambda x: p2f(x))
LFMbyRegion = LFM.melt(id_vars=['Date'])
LFMbyRegion.columns = ['Timestamp','District','Lfm']
RelevantPeriod = (LFMbyRegion['Timestamp']>'2021-01-01')&(LFMbyRegion['Timestamp']<'2021-02-09')
LFMPeriod = LFMbyRegion[RelevantPeriod]

FPI Table

In [251]:
FPITable = pd.DataFrame([(x_, y_) for x_ in set(LFMbyRegion['District']) for y_ in get_daterange(date(2021, 1, 21), date(2021, 2, 8), 
                                                                                   timedelta(days=1))])
FPITable.columns = ['District','Timestamp']
FPITable['Timestamp'] = pd.to_datetime(FPITable['Timestamp'])

LFM Load

In [252]:



dates =  []
for d in get_daterange(date(2021, 2, 9), date(2021, 2, 15), timedelta(days=1)):
    dates.append(d)

LFMForecast= pd.DataFrame(columns=['LFM','District','Timestamp'])
    
for district in set(LFMPeriod['District']):
    model = ARIMA(LFMPeriod[LFMPeriod['District']==district]['Lfm'], order=(7,1,1))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=7)
    tempdf = pd.DataFrame((predictions))
    tempdf['District'] = district

    tempdf['dates']= dates

    tempdf.columns = ['LFM','District','Timestamp']

    LFMForecast = LFMForecast.append(tempdf)
    
LFMForecast['Timestamp'] = pd.to_datetime(LFMForecast['Timestamp'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user

NDVI Load

Assumes relevant NDVI csv files are in same folder location

In [253]:
file_list = glob.glob('/home/ec2-user/SageMaker/MODISTerraNDVI/*.csv')

District = {'Black_Mtn_Ranch': 'NE',
   'Campo': 'ME' ,
   'Camp_Pendleton__Wheelan_Lake' : 'OC',
   'Case_Springs' : 'OC',
   'Fallbrook__Ammo_Dump' : 'NE',
   'Guejito_North': 'NE',
   'Guejito_West': 'NE',
   'Hellhole_Canyon': 'NE',
   'Highland_Valley_Rd': 'RA',
   'Honey_Springs_Rd': 'ME',
   'Mesa_Grande': 'RA',
   'Mission_Trail': 'BC',
   'Ortega_Highway': 'OC',
   'Santa_Rosa_Plateau' : 'NE',
   'Santa_Ysabel_North' : 'RA',
   'Sycamore_Canyon' :    'EA',
   'Viejas' : 'ME', 
   'Warner_Springs' :   'RA',
   'Witch_Creek' : 'RA'}

DateTable = pd.DataFrame([(x_, y_) for x_ in set(District.keys()) for y_ in get_daterange(date(2014, 1, 1), date(2021, 5, 10), 
                                                                                   timedelta(days=1))])
DateTable.columns = ['Station','Timestamp']
DateTable['District'] =  DateTable['Station'].apply(lambda x: District.get(x))
DateTable['Timestamp'] = pd.to_datetime(DateTable['Timestamp'])


Ndvi= pd.DataFrame(columns=['Station','District','Timestamp','NDVI','NDVILinearInterpolation','NDVIQuadraticInterpolation'])
for file in file_list:
     
    district = District.get(file.split('/')[-1].replace('.csv',''))
    station = file.split('/')[-1].replace('.csv','')
    temp = pd.read_csv(file)
    temp['# timestamp'] = pd.to_datetime(temp['# timestamp'])
    temp.columns= ['Timestamp', 'NDVI']
    temp['District'] = district
    temp['Station'] = station
    print(station)
    NdviAllDates = pd.merge(DateTable.loc[DateTable['District']==district],temp,how='left',left_on=['District','Station','Timestamp'],right_on=['District','Station','Timestamp'])
    NdviAllDates['NDVILinearInterpolation'] = NdviAllDates.interpolate(method='linear',limit_direction='forward')['NDVI']
    NdviAllDates['NDVIQuadraticInterpolation'] = NdviAllDates.interpolate(method='quadratic',limit_direction='forward')['NDVI']
    Ndvi = Ndvi.append(NdviAllDates)
NdviTable = pd.DataFrame(Ndvi.groupby(['District','Timestamp']).max())['NDVILinearInterpolation'].reset_index()
NdviTable.columns = ['District', 'Timestamp', 'NDVI']
RelevantPeriod = (NdviTable['Timestamp']>'2020-01-01') & (NdviTable['Timestamp']<'2021-02-09')
NDVIPeriod = NdviTable[RelevantPeriod]

Santa_Ysabel_North
Camp_Pendleton__Wheelan_Lake
Case_Springs
Guejito_West
Highland_Valley_Rd
Fallbrook__Ammo_Dump
Hellhole_Canyon
Viejas
Santa_Rosa_Plateau
Guejito_North
Sycamore_Canyon
Black_Mtn_Ranch
Campo
Mesa_Grande
Mission_Trails
Ortega_Highway
Warner_Springs
Honey_Springs_Rd
Witch_Creek


In [254]:
NDVIForecast= pd.DataFrame(columns=['NDVI','District','Timestamp'])
    
for district in set(NDVIPeriod['District']):
    model = ARIMA(NDVIPeriod[NDVIPeriod['District']==district]['NDVI'], order=(7,1,1))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=7)
    tempdf = pd.DataFrame((predictions))
    tempdf['District'] = district

    tempdf['dates']= dates

    tempdf.columns = ['NDVI','District','Timestamp']

    NDVIForecast = NDVIForecast.append(tempdf)

NDVIForecast['Timestamp'] = pd.to_datetime(NDVIForecast['Timestamp'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user

In [255]:
# file_list = glob.glob('/home/ec2-user/SageMaker/MODISTerraNDVI/*.csv')

# District = {'Black_Mtn_Ranch': 'NE',
#    'Campo': 'ME' ,
#    'Camp_Pendleton__Wheelan_Lake' : 'OC',
#    'Case_Springs' : 'OC',
#    'Fallbrook__Ammo_Dump' : 'NE',
#    'Guejito_North': 'NE',
#    'Guejito_West': 'NE',
#    'Hellhole_Canyon': 'NE',
#    'Highland_Valley_Rd': 'RA',
#    'Honey_Springs_Rd': 'ME',
#    'Mesa_Grande': 'RA',
#    'Mission_Trail': 'BC',
#    'Ortega_Highway': 'OC',
#    'Santa_Rosa_Plateau' : 'NE',
#    'Santa_Ysabel_North' : 'RA',
#    'Sycamore_Canyon' :    'EA',
#    'Viejas' : 'ME', 
#    'Warner_Springs' :   'RA',
#    'Witch_Creek' : 'RA'}

# DateTable = pd.DataFrame([(x_, y_) for x_ in set(District.values()) for y_ in get_daterange(date(2014, 1, 1), date(2021, 5, 10), 
#                                                                                    timedelta(days=1))])
# DateTable.columns = ['District','Timestamp']
# DateTable['Timestamp'] = pd.to_datetime(DateTable['Timestamp'])

# Ndvi= pd.DataFrame(columns=['Timestamp','NDVI','District'])
# for file in file_list:
#     district = District.get(file.split('/')[-1].replace('.csv',''))
#     temp = pd.read_csv(file,parse_dates=['# timestamp'])
#     temp.columns= ['Timestamp', 'NDVI']
#     temp['District'] = district
#     NdviAllDates = pd.merge(DateTable.loc[DateTable['District']==district],temp,how='left',left_on=['District','Timestamp'],right_on=['District','Timestamp'])
#     NdviAllDatesInterpolate = NdviAllDates.interpolate(method='linear',limit_direction='forward')
#     Ndvi = Ndvi.append(NdviAllDatesInterpolate)
    
# RelevantPeriod = (Ndvi['Timestamp']>'2020-01-01') & (Ndvi['Timestamp']<'2021-02-09')
# NDVIPeriod = Ndvi[RelevantPeriod]


# NDVIForecast= pd.DataFrame(columns=['NDVI','District','Timestamp'])
    
# for district in set(NDVIPeriod['District']):
#     model = ARIMA(NDVIPeriod[NDVIPeriod['District']==district]['NDVI'], order=(7,1,1))
#     model_fit = model.fit()
#     predictions = model_fit.forecast(steps=7)
#     tempdf = pd.DataFrame((predictions))
#     tempdf['District'] = district

#     tempdf['dates']= dates

#     tempdf.columns = ['NDVI','District','Timestamp']

#     NDVIForecast = NDVIForecast.append(tempdf)

# NDVIForecast['Timestamp'] = pd.to_datetime(NDVIForecast['Timestamp'])

MesoWestFuel Load
Requires Synoptic Data API Account

In [256]:
# start_time = time.time()

public_token = ""

ME_stations = ['DENC1','CMNC1','POTC1','ANEC1','BAJC1']
RA_stations = ['JULC1','RCHC1','OGVC1','PAMC1','PIHC1','GOSC1']
EA_stations = ['ANEC1','BAJC1']
NE_stations = ['PAMC1','PZAC1','VLCC1','PSQC1','CTGC1']
OC_stations = ['CAPC1','ALLC1']
NC_stations = ['WIRC1']
BC_stations = ['MIGC1','BVDC1']
CM_stations = ['MIGC1','BVDC1']

SD_stations = {'ME_stations': ME_stations, 'RA_stations': RA_stations, 'EA_stations': EA_stations, 'NE_stations': NE_stations, 'OC_stations': OC_stations, 'NC_stations': NC_stations, 'BC_stations': BC_stations, 'CM_stations': CM_stations}

Features = pd.DataFrame(columns=['Timestamp','Region','Station','date_time','fuel_temp_set_1','snow_interval_set_1','air_temp_set_1','relative_humidity_set_1','wind_speed_set_1','wind_direction_set_1','wind_gust_set_1','volt_set_1','solar_radiation_set_1','precip_accum_set_1','water_temp_set_1','peak_wind_speed_set_1','fuel_moisture_set_1','peak_wind_direction_set_1','dew_point_temperature_set_1d','wind_chill_set_1d','wind_cardinal_direction_set_1d','heat_index_set_1d'])
for i in SD_stations:
    for j in SD_stations[i]:
#         print(i,j)
        try:
            url = 'https://api.synopticdata.com/v2/stations/timeseries?stid='+j+'&obtimezone=local&start=202101010000&end=202102092359&vars=fuel_temp,snow_interval,air_temp,relative_humidity,wind_speed,wind_direction,wind_gust,volt,solar_radiation,precip_accum,water_temp,peak_wind_speed,fuel_moisture,peak_wind_direction,dew_point_temperature,wind_chill,wind_cardinal_direction,heat_index&token=' + public_token
            meso_west = requests.get(url).json()
            data = meso_west['STATION'][0]['OBSERVATIONS']
            df = pd.DataFrame(data, columns=['date_time','fuel_temp_set_1','snow_interval_set_1','air_temp_set_1','relative_humidity_set_1','wind_speed_set_1','wind_direction_set_1','wind_gust_set_1','volt_set_1','solar_radiation_set_1','precip_accum_set_1','water_temp_set_1','peak_wind_speed_set_1','fuel_moisture_set_1','peak_wind_direction_set_1','dew_point_temperature_set_1d','wind_chill_set_1d','wind_cardinal_direction_set_1d','heat_index_set_1d'])
            df['Timestamp'] = df['date_time'].apply(lambda x: str(x)[0:10])
            df['District'] = i
            df['Station'] = j
            Features = Features.append(df)
        except:
            print('Error')

MesoWestFuel = Features            
# print("--- %s seconds ---" % (time.time() - start_time))

In [257]:
MesoWestFuel['date_time'] = MesoWestFuel['date_time'].apply(lambda x: timeconversion(x))
MesoWestFuel['Timestamp'] = pd.to_datetime(MesoWestFuel['Timestamp'])
MesoWestFuel['District'] = MesoWestFuel['District'].apply(lambda x: x[:2])
MesoWestFuel['hour']  = MesoWestFuel['date_time'].apply(lambda x: x.hour)

In [258]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

# training_impute = np.array(MesoWestFuel[MesoWestFuel['date_time']>'2021-01-01 00:00:00-00:00'].drop(columns=['Timestamp', 'date_time','District','Station','snow_interval_set_1','water_temp_set_1','wind_chill_set_1d',
#                                                                                  'wind_cardinal_direction_set_1d','heat_index_set_1d']))

training_impute = MesoWestFuelTrainingImpute = np.array(MesoWestFuel[[
    'fuel_temp_set_1',
              'air_temp_set_1',
       'relative_humidity_set_1', 
    'wind_speed_set_1',
    'wind_gust_set_1',
    'precip_accum_set_1',
                                     'fuel_moisture_set_1',
              'peak_wind_speed_set_1','solar_radiation_set_1'
                                                   ]])

lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan,max_iter=10, verbose=2,imputation_order='roman',random_state=42)

MesoWestFuelImputationDF = pd.DataFrame(imp.fit_transform(training_impute))


MesoWestFuelImputationDF.columns = [
    'fuel_temp_set_1',
    'air_temp_set_1',
       'relative_humidity_set_1', 
    'wind_speed_set_1',
    'wind_gust_set_1',
    'precip_accum_set_1',
                                     'fuel_moisture_set_1',
'peak_wind_speed_set_1','solar_radiation_set_1'
                                   ]

# MesoWestFuelImputationDF.columns = ['fuel_temp_set_1', 'air_temp_set_1', 'relative_humidity_set_1',
#        'wind_speed_set_1', 'wind_direction_set_1', 'wind_gust_set_1',
#        'volt_set_1', 'solar_radiation_set_1', 'precip_accum_set_1',
#        'peak_wind_speed_set_1', 'fuel_moisture_set_1',
#        'peak_wind_direction_set_1', 'dew_point_temperature_set_1d']

MesoWestFueltemp = MesoWestFuel.reset_index()[['Timestamp','District','Station','date_time']].reset_index()
MesoWestFuelMICEDF = pd.merge(MesoWestFueltemp,MesoWestFuelImputationDF.reset_index(),how='left').drop(columns=['index'])
MesoWestFuelMICEDF['solar_radiation_set_1_flag'] = np.where(MesoWestFuelMICEDF['solar_radiation_set_1']>=400,1,0)

# MesoWestFueltemp = MesoWestFuel[MesoWestFuel['date_time']>'2019-01-01 00:00:00-00:00'].reset_index()[['Timestamp','District','Station','date_time']].reset_index()
# MesoWestFuelMICEDF = pd.merge(MesoWestFueltemp,MesoWestFuelImputationDF.reset_index(),how='left')
# MesoWestFuelMICEDF['solar_radiation_set_1_flag'] = np.where(MesoWestFuelMICEDF['solar_radiation_set_1']>=400,1,0)

[IterativeImputer] Completing matrix with shape (23983, 9)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.04
[IterativeImputer] Change: 25.5761664234497, scaled tolerance: 0.853 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.09
[IterativeImputer] Change: 1.3511359263974092, scaled tolerance: 0.853 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.13
[IterativeImputer] Change: 0.04673537498591962, scaled tolerance: 0.853 
[IterativeImputer] Early stopping criterion reached.


In [259]:
FPITableFuel = FPITable
for column in ['air_temp_set_1','relative_humidity_set_1','wind_gust_set_1', 'solar_radiation_set_1',
       'precip_accum_set_1', 'peak_wind_speed_set_1',
       'fuel_temp_set_1','fuel_moisture_set_1']:
    if column == 'fuel_moisture_set_1':
        temp = MesoWestFuelMICEDF.groupby(['Timestamp','District'])[column].min().reset_index()
        temp['District'] = temp['District'].apply(lambda x: x[:2])
        temp['Timestamp'] = pd.to_datetime(temp['Timestamp'])
    else:
        temp = MesoWestFuelMICEDF.groupby(['Timestamp','District'])[column].max().reset_index()
        temp['District'] = temp['District'].apply(lambda x: x[:2])
        temp['Timestamp'] = pd.to_datetime(temp['Timestamp'])
    
    FPITableFuel = pd.merge(FPITableFuel,temp, how='left',left_on=['Timestamp','District'], right_on=['Timestamp','District'])

In [260]:
MesoWestFuelForecast = pd.DataFrame()
for district in set(LFMPeriod['District']):     
    air_temp_set_1_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['air_temp_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    relative_humidity_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['relative_humidity_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    wind_gust_set_1_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['wind_gust_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    solar_radiation_set_1_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['solar_radiation_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    precip_accum_set_1_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['precip_accum_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    peak_wind_speed_set_1_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['peak_wind_speed_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    fuel_temp_set_1_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['fuel_temp_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    fuel_moisture_set_1_forecast = ARIMA(FPITableFuel[FPITableFuel['District']==district]['fuel_moisture_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    temp = pd.DataFrame({'Timestamp':dates,'District':district,'air_temp_set_1': air_temp_set_1_forecast, 'relative_humidity_set_1': relative_humidity_forecast, 'wind_gust_set_1': wind_gust_set_1_forecast, 'solar_radiation_set_1': solar_radiation_set_1_forecast, 'precip_accum_set_1': precip_accum_set_1_forecast, 'peak_wind_speed_set_1': peak_wind_speed_set_1_forecast, 'fuel_temp_set_1': fuel_temp_set_1_forecast, 'fuel_moisture_set_1':fuel_moisture_set_1_forecast})
    MesoWestFuelForecast = MesoWestFuelForecast.append(temp)
MesoWestFuelForecast['Timestamp'] = pd.to_datetime(MesoWestFuelForecast['Timestamp'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/base/model.py:568: Co

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/

In [261]:
DistrictSolarRadiationMaxSum = pd.DataFrame(MesoWestFuelMICEDF.groupby(['Timestamp','District','Station']).sum()['solar_radiation_set_1'].groupby(['Timestamp','District']).max()).reset_index()
DistrictSolarRadiationMaxSum.columns = ['Timestamp', 'District', 'max_sum_solar_radiation_set_1']

DistrictSolarRadiationAvg = pd.DataFrame(MesoWestFuelMICEDF.groupby(['Timestamp','District']).mean()['solar_radiation_set_1']).reset_index()
DistrictSolarRadiationAvg.columns = ['Timestamp', 'District', 'avg_solar_radiation_set_1']

DistrictSolarRadiationElevated = pd.DataFrame(MesoWestFuelMICEDF.groupby(['Timestamp','District','Station']).sum()['solar_radiation_set_1_flag'].groupby(['Timestamp','District']).max()).reset_index()
DistrictSolarRadiationElevated.columns = ['Timestamp', 'District', 'elevated_solar_radiation_set_1']

In [262]:
DistrictSolarRadiationMaxSumForecast= pd.DataFrame(columns=['max_sum_solar_radiation_set_1','District','Timestamp'])
    
for district in set(DistrictSolarRadiationMaxSum['District']):
    model = ARIMA(DistrictSolarRadiationMaxSum[DistrictSolarRadiationMaxSum['District']==district]['max_sum_solar_radiation_set_1'], order=(7,1,1))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=7)
    tempdf = pd.DataFrame((predictions))
    tempdf['District'] = district

    tempdf['dates']= dates

    tempdf.columns = ['max_sum_solar_radiation_set_1','District','Timestamp']
    DistrictSolarRadiationMaxSumForecast = DistrictSolarRadiationMaxSumForecast.append(tempdf)

DistrictSolarRadiationMaxSumForecast['Timestamp'] = pd.to_datetime(DistrictSolarRadiationMaxSumForecast['Timestamp'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user

In [263]:
DistrictSolarRadiationAvgForecast= pd.DataFrame(columns=['avg_solar_radiation_set_1','District','Timestamp'])
    
for district in set(DistrictSolarRadiationAvg['District']):
    model = ARIMA(DistrictSolarRadiationAvg[DistrictSolarRadiationAvg['District']==district]['avg_solar_radiation_set_1'], order=(7,1,1))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=7)
    tempdf = pd.DataFrame((predictions))
    tempdf['District'] = district

    tempdf['dates']= dates

    tempdf.columns = ['avg_solar_radiation_set_1','District','Timestamp']
    DistrictSolarRadiationAvgForecast = DistrictSolarRadiationAvgForecast.append(tempdf)

DistrictSolarRadiationAvgForecast['Timestamp'] = pd.to_datetime(DistrictSolarRadiationAvgForecast['Timestamp'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user

In [264]:
DistrictSolarRadiationElevatedForecast= pd.DataFrame(columns=['elevated_solar_radiation_set_1','District','Timestamp'])
    
for district in set(DistrictSolarRadiationElevated['District']):
    model = ARIMA(DistrictSolarRadiationElevated[DistrictSolarRadiationElevated['District']==district]['elevated_solar_radiation_set_1'], order=(7,1,1))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=7)
    tempdf = pd.DataFrame((predictions))
    tempdf['District'] = district

    tempdf['dates']= dates

    tempdf.columns = ['elevated_solar_radiation_set_1','District','Timestamp']
    DistrictSolarRadiationElevatedForecast = DistrictSolarRadiationElevatedForecast.append(tempdf)

DistrictSolarRadiationElevatedForecast['Timestamp'] = pd.to_datetime(DistrictSolarRadiationElevatedForecast['Timestamp'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user

In [265]:
MesoWestFuelForecast = pd.merge(MesoWestFuelForecast,DistrictSolarRadiationMaxSumForecast, how='left',left_on=['Timestamp','District'], right_on=['Timestamp','District'])
MesoWestFuelForecast = pd.merge(MesoWestFuelForecast,DistrictSolarRadiationAvgForecast, how='left',left_on=['Timestamp','District'], right_on=['Timestamp','District'])
MesoWestFuelForecast = pd.merge(MesoWestFuelForecast,DistrictSolarRadiationElevatedForecast , how='left',left_on=['Timestamp','District'], right_on=['Timestamp','District'])

MesoWestWind Load
Requires Synoptic Data API Account

In [266]:
import requests

public_token = ""

ME_stations_WSDD = ['KCZZ']
RA_stations_WSDD = ['KCZZ']
EA_stations_WSDD = ['KRNM']
NE_stations_WSDD = ['KRNM']
OC_stations_WSDD = ['KNFG']
NC_stations_WSDD = ['KCRQ']
BC_stations_WSDD = ['KMYF']
CM_stations_WSDD = ['KSDM']

SD_stations_WSDD = {'ME_stations': ME_stations_WSDD, 'RA_stations': RA_stations_WSDD, 'EA_stations': EA_stations_WSDD, 'NE_stations': NE_stations_WSDD, 'OC_stations': OC_stations_WSDD, 'NC_stations': NC_stations_WSDD, 'BC_stations': BC_stations_WSDD, 'CM_stations': CM_stations_WSDD}

Features_WSDD = pd.DataFrame(columns=['Timestamp','District','Station','date_time','air_temp_set_1','relative_humidity_set_1','wind_speed_set_1','wind_direction_set_1','wind_gust_set_1','peak_wind_speed_set_1','peak_wind_direction_set_1','dew_point_temperature_set_1d','volt_set_1','wind_chill_set_1d','wind_cardinal_direction_set_1d','heat_index_set_1d','altimeter_set_1','dew_point_temperature_set_1','sea_level_pressure_set_1','weather_cond_code_set_1','cloud_layer_3_code_set_1','pressure_tendency_set_1','precip_accum_one_hour_set_1','precip_accum_three_hour_set_1','cloud_layer_1_code_set_1','cloud_layer_2_code_set_1','precip_accum_six_hour_set_1','precip_accum_24_hour_set_1','visibility_set_1','metar_remark_set_1','metar_set_1','air_temp_high_6_hour_set_1','air_temp_low_6_hour_set_1','ceiling_set_1','pressure_change_code_set_1','air_temp_high_24_hour_set_1','air_temp_low_24_hour_set_1','cloud_layer_1_set_1d','cloud_layer_3_set_1d','cloud_layer_2_set_1d','weather_summary_set_1d','pressure_set_1d','sea_level_pressure_set_1d','weather_condition_set_1d'])
for i in SD_stations_WSDD:
    for j in SD_stations_WSDD[i]:
        print(i,j)
        try:
            url_wsdd = 'https://api.synopticdata.com/v2/stations/timeseries?stid='+j+'&obtimezone=local&start=202101010000&end=202102092359&token=' + public_token
            meso_west_wsdd = requests.get(url_wsdd).json()
            data_wsdd = meso_west_wsdd['STATION'][0]['OBSERVATIONS']
            df_wsdd = pd.DataFrame(data_wsdd, columns=['date_time','air_temp_set_1','relative_humidity_set_1','wind_speed_set_1','wind_direction_set_1','wind_gust_set_1','peak_wind_speed_set_1','peak_wind_direction_set_1','dew_point_temperature_set_1d','volt_set_1','wind_chill_set_1d','wind_cardinal_direction_set_1d','heat_index_set_1d','altimeter_set_1','dew_point_temperature_set_1','sea_level_pressure_set_1','weather_cond_code_set_1','cloud_layer_3_code_set_1','pressure_tendency_set_1','precip_accum_one_hour_set_1','precip_accum_three_hour_set_1','cloud_layer_1_code_set_1','cloud_layer_2_code_set_1','precip_accum_six_hour_set_1','precip_accum_24_hour_set_1','visibility_set_1','metar_remark_set_1','metar_set_1','air_temp_high_6_hour_set_1','air_temp_low_6_hour_set_1','ceiling_set_1','pressure_change_code_set_1','air_temp_high_24_hour_set_1','air_temp_low_24_hour_set_1','cloud_layer_1_set_1d','cloud_layer_3_set_1d','cloud_layer_2_set_1d','weather_summary_set_1d','pressure_set_1d','sea_level_pressure_set_1d','weather_condition_set_1d'])
            df_wsdd['Timestamp'] = df_wsdd['date_time'].apply(lambda x: str(x)[0:10])
            df_wsdd['District'] = i
            df_wsdd['Station'] = j
            #df1 = df.groupby(['Date','Region','Station']).max().reset_index()
            Features_WSDD = Features_WSDD.append(df_wsdd)
        except:
            print('Error')


ME_stations KCZZ
Error
RA_stations KCZZ
Error
EA_stations KRNM
NE_stations KRNM
OC_stations KNFG
NC_stations KCRQ
BC_stations KMYF
CM_stations KSDM


In [267]:
Features_WSDD['District'] = Features_WSDD['District'].apply(lambda x:x[:2])

In [268]:
Features_WSDD['District'] = Features_WSDD['District'].apply(lambda x : x[:2])
Features_WSDD['wind_speed_set_mph'] = Features_WSDD['wind_speed_set_1'].apply(lambda x: windconversion(x))

In [269]:
training_impute = Features_WSDD.select_dtypes(include='float64')

In [270]:
lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan,max_iter=10, verbose=2,imputation_order='roman',random_state=42)

MesoWestWindImputationDF = pd.DataFrame(imp.fit_transform(training_impute))
MesoWestWindImputationDF.columns = Features_WSDD.select_dtypes(include='float64').columns 
MesoWestWindtemp = Features_WSDD[['Timestamp','District','Station','date_time']]
MesoWestWindMICEDF = pd.merge(MesoWestWindtemp.reset_index(),MesoWestWindImputationDF.reset_index(),how='left').drop(columns=['index'])
MesoWestWindMICEDF['DewPointDepression'] = MesoWestWindMICEDF['air_temp_set_1']- MesoWestWindMICEDF['dew_point_temperature_set_1']

[IterativeImputer] Completing matrix with shape (60827, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.13
[IterativeImputer] Change: 382077108632.18134, scaled tolerance: 102.76335 
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.18
[IterativeImputer] Change: 1878493.8602843152, scaled tolerance: 102.76335 
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.33
[IterativeImputer] Change: 905103.1588848658, scaled tolerance: 102.76335 
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.38
[IterativeImputer] Change: 9032477.139557358, scaled tolerance: 102.76335 
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.52
[IterativeImputer] Change: 1693973.472081663, scaled tolerance: 102.76335 
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.57
[IterativeImputer] Change: 1908773.4887764018, scaled tolerance: 102.76335 
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.72
[IterativeImputer]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [271]:
FPITableWind = FPITable
for column in ['wind_speed_set_1','dew_point_temperature_set_1','dew_point_temperature_set_1d','DewPointDepression','wind_speed_set_mph']:
        temp = MesoWestWindMICEDF.groupby(['Timestamp','District'])[column].max().reset_index()
        temp['District'] = temp['District'].apply(lambda x: x[:2])
        temp['Timestamp'] = pd.to_datetime(temp['Timestamp'])
        
        FPITableWind = pd.merge(FPITableWind,temp, how='left',left_on=['Timestamp','District'], right_on=['Timestamp','District'])

In [272]:
MesoWestWindForecast = pd.DataFrame()
for district in {'BC', 'CM', 'EA', 'ME', 'NC', 'NE', 'OC', 'RA'} :     
    wind_speed_set_1_forecast = ARIMA(FPITableWind[FPITableWind['District']==district]['wind_speed_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    dew_point_temperature_set_1_forecast = ARIMA(FPITableWind[FPITableWind['District']==district]['dew_point_temperature_set_1'], order=(7,1,1)).fit().forecast(steps=7)
    dew_point_temperature_set_1d_forecast = ARIMA(FPITableWind[FPITableWind['District']==district]['dew_point_temperature_set_1d'], order=(7,1,1)).fit().forecast(steps=7)
    DewPointDepression_forecast = ARIMA(FPITableWind[FPITableWind['District']==district]['DewPointDepression'], order=(7,1,1)).fit().forecast(steps=7)
    wind_speed_set_mph_forecast = ARIMA(FPITableWind[FPITableWind['District']==district]['wind_speed_set_mph'], order=(7,1,1)).fit().forecast(steps=7)
    temp = pd.DataFrame({'Timestamp':dates,'District':district,'wind_speed_set_1': wind_speed_set_1_forecast, 'dew_point_temperature_set_1': dew_point_temperature_set_1_forecast , 'dew_point_temperature_set_1d': dew_point_temperature_set_1d_forecast, 'DewPointDepression': DewPointDepression_forecast, 'wind_speed_set_mph': wind_speed_set_mph_forecast, })
    MesoWestWindForecast = MesoWestWindForecast.append(temp)
MesoWestWindForecast['Timestamp'] = pd.to_datetime(MesoWestWindForecast['Timestamp'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. 

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/ec2-user

In [273]:
# from datetime import datetime, date, timedelta

# # dewpointtemp = pd.DataFrame(MesoWestWindMICEDF.groupby(['Timestamp','District'])[['DewPointDepression']].max()).reset_index()

# def get_daterange(start_date, end_date, step):
#     while start_date <= end_date:
#         yield start_date
#         start_date += step
# dates =  []
# for d in get_daterange(date(2021, 2, 9), date(2021, 2, 15), timedelta(days=1)):
#     dates.append(d)

# MesoWestWindForecast = pd.DataFrame()
    
# for district in set(dewpointtemp['District']):
#     air_temp_set_1_forecast = pd.DataFrame(ARIMA(dewpointtemp[dewpointtemp['District']==district]['DewPointDepression'], order=(1,1,1)).fit().forecast(steps=7))
#     air_temp_set_1_forecast['TimeStamp'] = pd.to_datetime(dates)
#     air_temp_set_1_forecast['District'] = district
#     air_temp_set_1_forecast.columns = ['DewPointDepression','Timestamp','District']
#     MesoWestWindForecast = MesoWestWindForecast.append(air_temp_set_1_forecast)
# MesoWestWindForecast['Timestamp'] = pd.to_datetime(MesoWestWindForecast['Timestamp'])

Combine Forecasted Tables

In [274]:
Forecast = pd.merge(MesoWestFuelForecast, MesoWestWindForecast , 'left',left_on=['Timestamp','District'],right_on=['Timestamp','District'])
Forecast = pd.merge(Forecast, NDVIForecast ,'left',left_on=['Timestamp','District'],right_on=['Timestamp','District'])
Forecast = pd.merge(Forecast, LFMForecast ,'left',left_on=['Timestamp','District'],right_on=['Timestamp','District'])
Forecast = Forecast.fillna(Forecast.mean())

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.


In [275]:
Forecast['DistrictCategory'] = Forecast['District'].astype('category').cat.codes
Forecast['Month'] = Forecast['Timestamp'].apply(lambda x: x.month)
Forecast['Season'] = Forecast['Timestamp'].apply(lambda x: seasonconversion(x))

In [276]:
Forecast[['air_temp_set_1', 'relative_humidity_set_1', 'wind_gust_set_1',
       'solar_radiation_set_1', 'precip_accum_set_1', 'peak_wind_speed_set_1',
       'fuel_temp_set_1', 'fuel_moisture_set_1',
       'max_sum_solar_radiation_set_1', 'avg_solar_radiation_set_1',
       'elevated_solar_radiation_set_1', 'wind_speed_set_1',
       'dew_point_temperature_set_1', 'dew_point_temperature_set_1d', 'NDVI',
       'LFM', 

               'Season', 'DistrictCategory', 'Month'
         ]]

,air_temp_set_1,relative_humidity_set_1,wind_gust_set_1,solar_radiation_set_1,precip_accum_set_1,peak_wind_speed_set_1,fuel_temp_set_1,fuel_moisture_set_1,max_sum_solar_radiation_set_1,avg_solar_radiation_set_1,elevated_solar_radiation_set_1,wind_speed_set_1,dew_point_temperature_set_1,dew_point_temperature_set_1d,NDVI,LFM,Season,DistrictCategory,Month
0,18.764128,95.732817,15.460197,656.054565,183.986966,15.460197,32.428852,6.494422,3201.189435,171.334929,4.484347,0.000000,0.000000,0.000000,0.704000,0.7,1,3,2
1,16.230045,96.386735,9.274828,668.453585,183.512298,9.274828,31.536250,6.847475,3993.067930,174.245713,5.212668,0.000000,0.000000,0.000000,0.704000,0.7,1,3,2
2,17.475848,96.878877,10.870075,738.165555,183.261913,10.870075,31.110494,7.289726,4351.679103,177.137376,5.644207,0.000000,0.000000,0.000000,0.704000,0.7,1,3,2
3,20.141367,96.040868,14.136286,789.179482,182.896369,14.136286,34.234792,7.558326,3929.250333,174.755674,5.361508,0.000000,0.000000,0.000000,0.704000,0.7,1,3,2
4,16.335713,96.651339,7.255047,778.968556,182.210410,7.255047,33.913823,7.765537,4663.878431,177.628631,5.683848,0.000000,0.000000,0.000000,0.704000,0.7,1,3,2
5,12.867616,96.789224,11.201147,813.110595,181.715109,11.201147,31.749919,7.697598,5215.887318,180.695325,6.206028,0.000000,0.000000,0.000000,0.704000,0.7,1,3,2
6,12.521367,96.439027,14.399693,857.130877,181.169999,14.399693,32.489811,7.525373,5268.418741,181.795862,6.458186,0.000000,0.000000,0.000000,0.704000,0.7,1,3,2
7,12.567410,98.160626,6.537671,556.592907,98.559874,6.537671,24.418087,12.089635,2688.886800,130.028718,3.496231,4.676275,7.657170,8.158172,0.655448,0.7,1,4,2
8,14.112441,100.437533,7.043533,559.168819,98.606012,7.043533,23.418465,10.089561,3189.643144,132.291138,2.863889,5.143926,6.558261,8.309971,0.655448,0.7,1,4,2
9,16.735292,99.374098,6.369776,564.793640,98.649853,6.369776,27.909047,9.057910,3027.395403,125.426241,2.765428,5.679724,6.767833,9.120872,0.655448,0.7,1,4,2


In [277]:
# Forecast1 = Forecast.dropna()

In [278]:
random_forest_pkl_filename = 'random_forest_regressor.pkl'
# Loading the saved decision tree model pickle
random_forest_model_pkl = open(random_forest_pkl_filename, 'rb')
random_forest_regressor_model = pickle.load(random_forest_model_pkl)

In [280]:
xg_boost_class_pkl_filename = 'xg_boost_class.pkl'
# Open the file to save as pkl file
xg_boost_class_model_pkl = open(xg_boost_class_pkl_filename, 'rb')
xgb_class = pickle.load(xg_boost_class_model_pkl)

In [289]:
Forecast['FPIPrediction'] = random_forest_regressor_model.predict(Forecast[['air_temp_set_1', 'relative_humidity_set_1', 'wind_gust_set_1',
       'solar_radiation_set_1', 'precip_accum_set_1', 'peak_wind_speed_set_1',
       'fuel_temp_set_1', 'fuel_moisture_set_1',
       'max_sum_solar_radiation_set_1', 'avg_solar_radiation_set_1',
       'elevated_solar_radiation_set_1', 'wind_speed_set_1',
       'dew_point_temperature_set_1', 'dew_point_temperature_set_1d', 'NDVI',
       'LFM', 

               'Season', 'DistrictCategory', 'Month'
         ]])

In [291]:
Forecast['FPIRiskEvent'] =  xgb_class.predict(

Forecast[['air_temp_set_1', 'relative_humidity_set_1', 'wind_gust_set_1',
       'solar_radiation_set_1', 'precip_accum_set_1', 'peak_wind_speed_set_1',
       'fuel_temp_set_1', 'fuel_moisture_set_1',
       'elevated_solar_radiation_set_1', 'wind_speed_set_1',
       'dew_point_temperature_set_1', 'dew_point_temperature_set_1d', 'NDVI',
       'LFM', 'Season', 'DistrictCategory', 'Month']]

)

In [310]:
Forecast = pd.merge(Forecast,DistrictLocation,how='left')

In [283]:
totaltime = time.time() - start_time
print(totaltime)

81.967693567276


In [312]:
Forecast.to_csv('ModeledPredictions.csv')